In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Perceptron, SGDClassifier
from sklearn.model_selection import GridSearchCV
from BorutaShap import BorutaShap
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import cross_val_predict
from xgboost import XGBClassifier  
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

RANDOM_STATE = 42
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")


# all DEGS

In [2]:
import json
df_train = pd.read_csv('data/df_train.txt', sep='\t')
df_test = pd.read_csv('data/df_test.txt', sep='\t')
with open('data/train_deg_ml_list.json', 'r') as file:
    total_degs_disease = json.load(file)

## delete red blood cell
genes_to_remove = {'HBA1', 'HBB', 'HBA2'}

for key in total_degs_disease:
    total_degs_disease[key] = [
        gene for gene in total_degs_disease[key]
        if gene not in genes_to_remove
    ]

In [4]:
df_disease_train = {}
for key in total_degs_disease.keys():
    filtered_df = df_train[['ID','disease_code_level2']+total_degs_disease[key]]
    filtered_df['disease_code_level2'] = filtered_df['disease_code_level2'].apply(lambda x: 1 if x == key else 0)
    filtered_df.set_index('ID', inplace=True)
    df_disease_train[key] = filtered_df
    
df_disease_test = {}
for key in total_degs_disease.keys():
    filtered_df = df_test[['ID','disease_code_level2']+total_degs_disease[key]]
    filtered_df['disease_code_level2'] = filtered_df['disease_code_level2'].apply(lambda x: 1 if x == key else 0)
    filtered_df.set_index('ID', inplace=True)
    df_disease_test[key] = filtered_df

In [6]:
# predict using all genes
ml_res_all_genes = {}
cv = StratifiedKFold(shuffle=True, random_state=RANDOM_STATE, n_splits=5)


for disease in df_disease_train.keys():
    x_train, y_train = df_disease_train[disease].iloc[:,1:].values, np.array(df_disease_train[disease]['disease_code_level2'].astype(int))
    x_test, y_test = df_disease_test[disease].iloc[:,1:].values, np.array(df_disease_test[disease]['disease_code_level2'].astype(int))
    
    pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                           ( "XGBoost", XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=RANDOM_STATE))])

    pipeline.fit(x_train, y_train)
    y_predictions = pipeline.predict(x_test)
    accuracy = accuracy_score(y_test, y_predictions)
    f1_scores = f1_score(y_test, y_predictions, average='weighted')
    ml_res_all_genes[disease] = {'accuracy': accuracy, 'f1_score': f1_scores}




In [8]:
ml_res_all_genes

{'CTRL': {'accuracy': 0.9387755102040817, 'f1_score': 0.9292882599550562},
 'AS': {'accuracy': 0.978021978021978, 'f1_score': 0.9761824353241999},
 'PS': {'accuracy': 0.9262166405023547, 'f1_score': 0.9126950990715589},
 'CAD': {'accuracy': 0.978021978021978, 'f1_score': 0.975189295156562},
 'RA': {'accuracy': 0.9623233908948194, 'f1_score': 0.9546353347360721},
 'BC': {'accuracy': 0.9858712715855573, 'f1_score': 0.9850738267203158},
 'DM': {'accuracy': 0.9497645211930926, 'f1_score': 0.9315368586033093},
 'EP': {'accuracy': 0.978021978021978, 'f1_score': 0.973696478959637},
 'GBS': {'accuracy': 0.989010989010989, 'f1_score': 0.9855449380389998},
 'PCOS': {'accuracy': 0.9686028257456829, 'f1_score': 0.9583894898548092},
 'SCZ': {'accuracy': 0.989010989010989, 'f1_score': 0.9855449380389998},
 'CC': {'accuracy': 0.9937205651491365, 'f1_score': 0.993207326007326},
 'SEP': {'accuracy': 0.989010989010989, 'f1_score': 0.9877246988180747},
 'BD': {'accuracy': 0.9858712715855573, 'f1_score': 

# shap calculation

In [12]:
import json    
from sklearn.ensemble import RandomForestClassifier
import xgboost
import shap
import matplotlib.pyplot as plt
shap_top10 = {}
shap_top100 = {}

for disease in df_disease_train.keys():
    X_shap = df_disease_train[disease].drop(['disease_code_level2'],axis = 1)
    y_shap = df_disease_train[disease]['disease_code_level2']
    y_shap = y_shap.astype(int)

    model = xgboost.XGBRegressor().fit(X_shap, y_shap)
    explainer = shap.Explainer(model)
    shap_values = explainer(X_shap)
    
    
    ## retrieve value
    feature_names = X_shap.columns
    mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
    features_with_shap = [(feature, float(value)) for feature, value in zip(feature_names, mean_abs_shap)]
    sorted_features = sorted(features_with_shap, key=lambda x: x[1], reverse=True)

    top_10_features = [feature for feature, _ in sorted_features[:10]]
    shap_top10[disease] = top_10_features
    top_100_features = [feature for feature, _ in sorted_features[:100]]
    shap_top100[disease] = top_10_features

    


In [25]:
with open('results/shap_top10.json', 'w') as json_file:
    json.dump(shap_top10, json_file, indent=4)

# shap 10 prediction

In [16]:
df_disease_train_shap10 = {}
for key in df_disease_train.keys():
    df_disease_train_shap10[key] = df_disease_train[key][['disease_code_level2']+shap_top10[key]]

df_disease_test_shap10 = {}
for key in df_disease_test.keys():
    df_disease_test_shap10[key] = df_disease_test[key][['disease_code_level2']+shap_top10[key]]


ml_res_shap10_genes = {}

for disease in df_disease_train_shap10.keys():
    x_train, y_train = df_disease_train_shap10[disease].iloc[:,1:].values, np.array(df_disease_train_shap10[disease]['disease_code_level2'].astype(int))
    x_test, y_test = df_disease_test_shap10[disease].iloc[:,1:].values, np.array(df_disease_test_shap10[disease]['disease_code_level2'].astype(int))
    
    pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                           ( "XGBoost", XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=RANDOM_STATE))])

    pipeline.fit(x_train, y_train)
    y_predictions = pipeline.predict(x_test)
    accuracy = accuracy_score(y_test, y_predictions)
    f1_scores = f1_score(y_test, y_predictions, average='weighted')
    ml_res_shap10_genes[disease] = {'accuracy': accuracy, 'f1_score': f1_scores}




# shap 100 prediction

In [18]:
df_disease_train_shap100 = {}
for key in df_disease_train.keys():
    df_disease_train_shap100[key] = df_disease_train[key][['disease_code_level2']+shap_top100[key]]

df_disease_test_shap100 = {}
for key in df_disease_test.keys():
    df_disease_test_shap100[key] = df_disease_test[key][['disease_code_level2']+shap_top100[key]]


ml_res_shap100_genes = {}

for disease in df_disease_train_shap10.keys():
    x_train, y_train = df_disease_train_shap100[disease].iloc[:,1:].values, np.array(df_disease_train_shap100[disease]['disease_code_level2'].astype(int))
    x_test, y_test = df_disease_test_shap100[disease].iloc[:,1:].values, np.array(df_disease_test_shap100[disease]['disease_code_level2'].astype(int))
    
    pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                           ( "XGBoost", XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=RANDOM_STATE))])

    pipeline.fit(x_train, y_train)
    y_predictions = pipeline.predict(x_test)
    accuracy = accuracy_score(y_test, y_predictions)
    f1_scores = f1_score(y_test, y_predictions, average='weighted')
    ml_res_shap100_genes[disease] = {'accuracy': accuracy, 'f1_score': f1_scores}

# multi class

In [20]:
shap_gene = set(gene for genes in shap_top10.values() for gene in genes)
print(len(shap_gene))

Disease_list = total_degs_disease.keys()
disease_mapping = {disease: i for i, disease in enumerate(Disease_list)}


245


In [21]:
x_train, y_train = df_train[list(shap_gene)].values, np.array([disease_mapping[disease] for disease in df_train['disease_code_level2'].tolist()], dtype=int)
x_test, y_test = df_test[list(shap_gene)].values, np.array([disease_mapping[disease] for disease in df_test['disease_code_level2'].tolist()], dtype=int)

# oversample
from imblearn.over_sampling import SMOTE, SVMSMOTE

# SMOTE
sm = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = sm.fit_resample(x_train, y_train)

In [17]:
import numpy as np
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier  
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

RANDOM_STATE = 42

# Define classifiers
classifiers = {
    "K-neighbours": KNeighborsClassifier(),
    "Decision tree": DecisionTreeClassifier(),
    "Random forest": RandomForestClassifier(random_state=RANDOM_STATE),
    "Extremely random forest": ExtraTreesClassifier(random_state=RANDOM_STATE),
    "Adaboost": AdaBoostClassifier(),
    "Bagging": BaggingClassifier(random_state=RANDOM_STATE),
    "MLP": MLPClassifier(max_iter=2000, hidden_layer_sizes=(20, 20)),
    "SVM (polynomial)": SVC(kernel="poly",probability=True),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=RANDOM_STATE)
}

clf_best = None
clf_name_best = ""
score_best = 0

cv = StratifiedKFold(shuffle=True, random_state=RANDOM_STATE, n_splits=10)

# Separate features
#categorical_columns = [0]  
numerical_columns = list(range(0, x_train.shape[1])) 

# Iterate through the classifiers
for clf_name, clf in classifiers.items():
    
    # Preprocessing for numerical and categorical data
    #categorical_transformer = Pipeline(steps=[
    #    ('encoder', OrdinalEncoder())
    #])
    
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numerical_columns),
       # ('cat', categorical_transformer, categorical_columns)
    ])

    # Create pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', clf)
    ])
    
    # Calculate cross-validation score
    cv_score = np.mean(cross_val_score(pipeline, X_resampled_smote, y_resampled_smote, cv=cv))

    print(f"{clf_name} scored {cv_score} during cross-validation")
    
    # Check if the current classifier is the best
    if cv_score > score_best:
        score_best = cv_score
        clf_best = pipeline
        clf_name_best = clf_name

print(f"\nBest classifier: {clf_name_best} with a score of {score_best}")

K-neighbours scored 0.8833656443667566 during cross-validation
Decision tree scored 0.8079151438105832 during cross-validation
Random forest scored 0.9459404629482494 during cross-validation
Extremely random forest scored 0.9440982043540442 during cross-validation
Adaboost scored 0.07384501297738229 during cross-validation
Bagging scored 0.904762434450977 during cross-validation
MLP scored 0.943869378674718 during cross-validation
SVM (polynomial) scored 0.868183696170348 during cross-validation
XGBoost scored 0.9475517771068382 during cross-validation

Best classifier: XGBoost with a score of 0.9475517771068382


## xgbboost

In [22]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import uniform, randint

cv = StratifiedKFold(shuffle=True, random_state=RANDOM_STATE, n_splits=10)

numerical_columns = list(range(0, x_train.shape[1])) 

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_columns)
])


pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ( 'XGBoost', XGBClassifier(eval_metric='logloss', random_state=RANDOM_STATE))])

params = {
    'XGBoost__n_estimators': randint(100, 400),              
    'XGBoost__max_depth': randint(3, 8),                      
    'XGBoost__learning_rate': uniform(0.01, 0.1),            
    'XGBoost__subsample': uniform(0.6, 0.4),               
    'XGBoost__colsample_bytree': uniform(0.6, 0.4)          
}

XGBoost_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=params,
    n_iter=50,
    scoring='accuracy',  
    n_jobs=-1,
    cv=cv,
    verbose=1,
    random_state=RANDOM_STATE
)

XGBoost_search.fit(X_resampled_smote, y_resampled_smote)

print("Best accuracy:", XGBoost_search.best_score_)
print("Best Params:", XGBoost_search.best_params_)


Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best accuracy: 0.9629625509825731
Best Params: {'XGBoost__colsample_bytree': 0.672894435115225, 'XGBoost__learning_rate': 0.08553614103176525, 'XGBoost__max_depth': 4, 'XGBoost__n_estimators': 359, 'XGBoost__subsample': 0.6739417822102108}


In [23]:
print("Best accuracy:", XGBoost_search.best_score_)
print("Best Params:", XGBoost_search.best_params_)


Best accuracy: 0.9629625509825731
Best Params: {'XGBoost__colsample_bytree': 0.672894435115225, 'XGBoost__learning_rate': 0.08553614103176525, 'XGBoost__max_depth': 4, 'XGBoost__n_estimators': 359, 'XGBoost__subsample': 0.6739417822102108}


In [24]:
best_model = XGBoost_search.best_estimator_
y_pred = best_model.predict(x_test)
y_proba = best_model.predict_proba(x_test)[:, 1]

from sklearn.metrics import accuracy_score, f1_score, classification_report,balanced_accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred, average='weighted'))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

report_dict = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()


Accuracy: 0.7880690737833596
F1: 0.7876740484295508
Balanced Accuracy: 0.7946681871062012
              precision    recall  f1-score   support

           0       0.78      0.86      0.81        69
           1       0.84      0.78      0.81        27
           2       0.84      0.72      0.78        65
           3       0.83      0.88      0.85        33
           4       0.72      0.67      0.69        27
           5       0.93      0.85      0.89        33
           6       0.71      0.71      0.71        35
           7       0.67      0.71      0.69        14
           8       0.53      0.89      0.67         9
           9       0.75      0.72      0.73        25
          10       1.00      0.89      0.94         9
          11       0.71      0.86      0.77        14
          12       0.61      0.85      0.71        13
          13       0.67      0.44      0.53         9
          14       0.86      0.50      0.63        12
          15       0.93      0.93      0.93  